In [ ]:
#hide

%pip install pandas
%pip install numpy
%pip install scikit-learn==1.5.2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [293]:
import pandas as pd #type: ignore
import numpy as np #type: ignore

In [294]:
df_classification = pd.read_csv("./train_taxi.csv")

In [295]:
df_classification.head()

,ID,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
0,T03315,2.0,2021-07-02 17:19:11,2021-07-02 17:40:02,N,1.0,152,142,1.0,4.30,...,1.0,0.5,5.00,0.00,NaN,0.3,27.55,1.0,2.75,1.0
1,T07720,2.0,2021-07-05 22:48:04,2021-07-05 23:12:02,N,1.0,93,107,2.0,13.52,...,0.5,0.5,12.15,6.55,NaN,0.3,60.75,1.0,2.75,1.0
2,T09695,2.0,2021-07-07 06:29:43,2021-07-07 06:38:42,N,1.0,74,75,1.0,1.55,...,0.0,0.5,0.00,0.00,NaN,0.3,8.80,1.0,0.00,2.0
3,T08802,2.0,2021-07-06 15:33:29,2021-07-06 15:43:25,N,1.0,226,129,1.0,1.22,...,0.0,0.5,0.00,0.00,NaN,0.3,8.80,1.0,0.00,2.0
4,T01413,2.0,2021-07-01 17:09:07,2021-07-01 18:03:12,N,1.0,55,86,1.0,14.23,...,1.0,0.5,0.00,2.45,NaN,0.3,49.75,1.0,0.00,1.0


In [296]:
df_classification = df_classification.drop(columns=['ehail_fee'])

In [297]:
df_classification = df_classification[df_classification['payment_type'].isin([1,2])]

In [298]:
df_classification =df_classification.drop(columns=['ID'])

In [299]:
from sklearn.preprocessing import LabelEncoder #type: ignore

label_encoder = LabelEncoder()

df_classification["store_and_fwd_flag"] = label_encoder.fit_transform(df_classification["store_and_fwd_flag"])

In [300]:
df_classification['pickup_datetime'] = pd.to_datetime(df_classification['lpep_pickup_datetime']) 
df_classification['dropoff_datetime'] = pd.to_datetime(df_classification['lpep_dropoff_datetime'])

In [301]:
df_classification['pickup_hour'] = df_classification['pickup_datetime'].dt.hour
df_classification['pickup_day'] = df_classification['pickup_datetime'].dt.dayofweek
df_classification['pickup_month'] = df_classification['pickup_datetime'].dt.month
df_classification['dropoff_hour'] = df_classification['dropoff_datetime'].dt.hour
df_classification['dropoff_day'] = df_classification['dropoff_datetime'].dt.dayofweek
df_classification['dropoff_month'] = df_classification['dropoff_datetime'].dt.month

In [302]:
df_classification['trip_duration'] = (df_classification['dropoff_datetime'] - df_classification['pickup_datetime']).dt.total_seconds() / 60

In [303]:
def encode_cyclical(feature, max_val):
    df_classification[f'{feature}_sin'] = np.sin(2 * np.pi * df_classification[feature] / max_val)
    df_classification[f'{feature}_cos'] = np.cos(2 * np.pi * df_classification[feature] / max_val)

In [304]:
encode_cyclical('pickup_hour', 24)
encode_cyclical('dropoff_hour', 24)
encode_cyclical('pickup_month', 12)
encode_cyclical('dropoff_month', 12)

In [305]:
df_classification = df_classification.drop(columns=['pickup_hour', 'pickup_month', 'dropoff_hour', 'dropoff_month', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'pickup_datetime', 'dropoff_datetime'], axis=1)

In [306]:
df_classification.head()

,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,...,dropoff_day,trip_duration,pickup_hour_sin,pickup_hour_cos,dropoff_hour_sin,dropoff_hour_cos,pickup_month_sin,pickup_month_cos,dropoff_month_sin,dropoff_month_cos
0,2.0,0,1.0,152,142,1.0,4.30,18.0,1.0,0.5,...,4,20.850000,-0.965926,-2.588190e-01,-0.965926,-2.588190e-01,-0.5,-0.866025,-0.5,-0.866025
1,2.0,0,1.0,93,107,2.0,13.52,38.0,0.5,0.5,...,0,23.966667,-0.500000,8.660254e-01,-0.258819,9.659258e-01,-0.5,-0.866025,-0.5,-0.866025
2,2.0,0,1.0,74,75,1.0,1.55,8.0,0.0,0.5,...,2,8.983333,1.000000,6.123234e-17,1.000000,6.123234e-17,-0.5,-0.866025,-0.5,-0.866025
3,2.0,0,1.0,226,129,1.0,1.22,8.0,0.0,0.5,...,1,9.933333,-0.707107,-7.071068e-01,-0.707107,-7.071068e-01,-0.5,-0.866025,-0.5,-0.866025
4,2.0,0,1.0,55,86,1.0,14.23,45.5,1.0,0.5,...,3,54.083333,-0.965926,-2.588190e-01,-1.000000,-1.836970e-16,-0.5,-0.866025,-0.5,-0.866025


In [307]:
pickup_day_one_hot = pd.get_dummies(df_classification['pickup_day'], prefix='pickup_day', drop_first=True).astype(int)
dropoff_day_one_hot = pd.get_dummies(df_classification['dropoff_day'], prefix='dropoff_day', drop_first=True).astype(int)

In [308]:
df_classification = pd.concat([df_classification, pickup_day_one_hot, dropoff_day_one_hot], axis=1) 

In [309]:
df_classification['payment_type'] = df_classification['payment_type'].replace({
    1: 0,
    2: 1
})

In [310]:
df_classification.columns

Index(['VendorID', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID',
       'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'trip_type',
       'congestion_surcharge', 'payment_type', 'pickup_day', 'dropoff_day',
       'trip_duration', 'pickup_hour_sin', 'pickup_hour_cos',
       'dropoff_hour_sin', 'dropoff_hour_cos', 'pickup_month_sin',
       'pickup_month_cos', 'dropoff_month_sin', 'dropoff_month_cos',
       'pickup_day_1', 'pickup_day_2', 'pickup_day_3', 'pickup_day_4',
       'pickup_day_5', 'pickup_day_6', 'dropoff_day_1', 'dropoff_day_2',
       'dropoff_day_3', 'dropoff_day_4', 'dropoff_day_5', 'dropoff_day_6'],
      dtype='object')

In [311]:
import tensorflow as tf # type: ignore
from tensorflow import keras # type: ignore
from tensorflow.keras import layers # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.preprocessing import StandardScaler # type: ignore
from sklearn.metrics import f1_score # type: ignore

In [312]:
numeric_features = ['passenger_count', 'trip_distance', 'fare_amount',
                    'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
                    'improvement_surcharge', 'total_amount', 'trip_duration',
                    'congestion_surcharge']

other_features = ['VendorID', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID',
                  'payment_type', 'DOLocationID', 'trip_type', 'pickup_hour_sin', 'pickup_hour_cos',
                  'dropoff_hour_sin', 'dropoff_hour_cos', 'pickup_month_sin',
                  'pickup_month_cos', 'dropoff_month_sin', 'dropoff_month_cos',
                  'pickup_day_1', 'pickup_day_2', 'pickup_day_3', 'pickup_day_4',
                  'pickup_day_5', 'pickup_day_6', 'dropoff_day_1', 'dropoff_day_2',
                  'dropoff_day_3', 'dropoff_day_4', 'dropoff_day_5', 'dropoff_day_6']

numeric_df = df_classification[numeric_features]
other_df = df_classification[other_features]
scaler = StandardScaler()
scaled_numeric = scaler.fit_transform(numeric_df)
scaled_numeric_df = pd.DataFrame(scaled_numeric, columns=numeric_features, index=df_classification.index)
df_scaled = pd.concat([scaled_numeric_df, other_df], axis=1)

df_scaled.head()

,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration,congestion_surcharge,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,payment_type,DOLocationID,trip_type
0,-0.295471,0.177742,0.199348,1.063407,0.210617,1.494335,-0.234470,0.063637,0.570267,-0.018515,1.883270,2.0,0,1.0,152,0.0,142,1.0
1,0.790341,2.176763,1.575757,0.266442,0.210617,4.372719,4.110185,0.063637,2.632497,0.014351,1.883270,2.0,0,1.0,93,0.0,107,1.0
2,-0.295471,-0.418496,-0.488856,-0.530523,0.210617,-0.518521,-0.234470,0.063637,-0.594396,-0.143653,-0.530991,2.0,0,1.0,74,1.0,75,1.0
3,-0.295471,-0.490044,-0.488856,-0.530523,0.210617,-0.518521,-0.234470,0.063637,-0.594396,-0.133635,-0.530991,2.0,0,1.0,226,1.0,129,1.0
4,-0.295471,2.330701,2.091910,1.063407,0.210617,-0.518521,1.390630,0.063637,1.949228,0.331940,-0.530991,2.0,0,1.0,55,0.0,86,1.0


In [313]:
X = df_scaled.drop(columns='payment_type', axis=1)
y = df_scaled['payment_type']

In [314]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [315]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.F1Score(name='f1_score', threshold=0.5)])

In [316]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

Epoch 1/100
179/179 [==============================] - 1s 4ms/step - loss: 4.3963 - accuracy: 0.5147 - f1_score: 0.3974 - val_loss: 0.6584 - val_accuracy: 0.6331 - val_f1_score: 0.0332
Epoch 2/100
179/179 [==============================] - 0s 3ms/step - loss: 0.9700 - accuracy: 0.5355 - f1_score: 0.3727 - val_loss: 0.6865 - val_accuracy: 0.6346 - val_f1_score: 0.0492
Epoch 3/100
179/179 [==============================] - 0s 3ms/step - loss: 0.7443 - accuracy: 0.5660 - f1_score: 0.2672 - val_loss: 0.6714 - val_accuracy: 0.6283 - val_f1_score: 0.0000e+00
Epoch 4/100
179/179 [==============================] - 1s 3ms/step - loss: 0.7092 - accuracy: 0.5784 - f1_score: 0.1101 - val_loss: 0.6763 - val_accuracy: 0.6283 - val_f1_score: 0.0000e+00
Epoch 5/100
179/179 [==============================] - 1s 3ms/step - loss: 0.7006 - accuracy: 0.5772 - f1_score: 0.0765 - val_loss: 0.6648 - val_accuracy: 0.6283 - val_f1_score: 0.0000e+00
Epoch 6/100
179/179 [==============================] - 1s 3ms/s

KeyboardInterrupt: 

In [291]:
y_pred = (model.predict(X_test) > 0.5).astype("int64")
y_pred = y_pred.flatten()
y_test = y_test.astype("int64")
f1 = f1_score(y_test, y_pred, average="binary")
print(f1)


50/50 [==============================] - 0s 1000us/step
0.8926701570680629
